In [252]:
#IMPORTS

from sqlalchemy import create_engine

import re
import pandas as pd
import numpy as np
from plotnine import *
from pandas import DataFrame, read_csv
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

In [253]:
#MYSQL CONNECTIONS

mysql_user = "root"
mysql_password = "localhost"
mysql_database = "airline_passenger_satisfaction"
mysql_host = "127.0.0.1"
port = 3306

sqlEngine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_database}',pool_recycle=port)
dbConnection = sqlEngine.connect()

In [1]:
#FUNCTIONS


def stripChar(x):
    try:
        return re.sub("[()\"]","",x)
    except:
        return np.nan



In [255]:
#MAIN_FILE INTO DATAFRAME

df_original = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Data/City/airport-codes.csv",header=0,encoding = 'unicode_escape')

#DROP EMPTY ROWS
df_original = df_original.dropna()

#DROP DUPLICATES
df_original = df_original.drop_duplicates(keep='first')

#REPLACE SPACE WITH UNDERSCORE FOR COLUMNS
df_original.columns = df_original.columns.str.replace(' ', '_')

#LOWERCASE ALL THE COLUMN NAMES
df_original.columns = [x.lower() for x in df_original.columns]

#RENAME COLUMN VALUES
df_original.rename(columns={'country_abbrev.':'country_code','airport_code':'iata_airport'}, inplace=True)

#Removing duplicate iata values
#df_original.drop_duplicates(subset = ['iata_airport'], keep = 'first', inplace = True) 
#Removing duplicate iata values
df_original.drop_duplicates(subset = ['country_code','country_name'], keep = 'first', inplace = True) 
df_original.drop_duplicates(subset = ['world_area_code'], keep = 'first', inplace = True)

#df_original


In [256]:
#INPUT TABLE AIRPORT TABLE

df_airport = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Data/Airport/airport_data_02.txt",header = 0)


#df_airport.loc[df_airport['country_code'].isin('IN')]

# #DROP UNWANTED COLUMNS
# df_airport=df_airport.drop(['url','city','county','state','code'], axis=1)

#RENAMING COLUMN NAMES
df_airport.rename(columns={'region_name':'state','airport':'airport_name','iata':'iata_airport','icao':'icao_airport'}, inplace=True)

# df_airport=df_airport.drop(['url','city','county','state','code'], axis=1)

#Removing duplicate iata values
df_airport.drop_duplicates(subset = ['iata_airport'], keep = 'first', inplace = True) 
df_airport=df_airport.drop_duplicates(keep='first')

df_airport = df_airport[['iata_airport','icao_airport','airport_name','state','country_code','latitude','longitude']]

len(list(df_airport['country_code'].unique()))

#df_airport

#WRITING THE TABLE TO DATABASE
#df_airport.to_sql(con=dbConnection,name='AIRPORT',if_exists='replace',index=False)

#print("Airport table added to database successfully")

225

In [257]:
#READING CITY FILE

df_city= pd.read_csv("/Users/shreyajaiswal/Downloads/airport-codes-2.csv",header=0)
df_city = df_city.dropna()
df_city=df_city.rename(columns={'Airport Code':'iata_airport'})
#df_city

In [258]:
#df_review_merge = merge_df.drop_duplicates(keep='first')
merge_df = pd.merge(df_airport, df_city, on='iata_airport', how='inner')
merge_df = merge_df.drop(['country_code','Airport Name'],axis=1)
merge_df = merge_df.rename(columns={'Country Abbrev.':'country_code','World Area Code':'world_area_code'})
merge_df = merge_df.drop_duplicates(keep='first')
merge_df['area_code_id'] = merge_df['country_code']+merge_df['world_area_code']
merge_df

#DROP EMPTY ROWS
merge_df = merge_df.dropna()

#REPLACE SPACE WITH UNDERSCORE FOR COLUMNS
merge_df.columns = merge_df.columns.str.replace(' ', '_')

#LOWERCASE ALL THE COLUMN NAMES
merge_df.columns = [x.lower() for x in merge_df.columns]

#REMOVE UNWANTED CHARACTERS FROM ROWS
merge_df.country_name = merge_df.country_name.apply(lambda x: stripChar(x))

#merge_df

#len(list(merge_df['iata_airport'].unique()))

In [259]:
#POPULATE COUNTRY TABLE

df_country= pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Data/Country code/country_code.txt",header=0)

df_country.rename(columns={'Name':'country_name','Code':'country_code'}, inplace=True)


#REPLACE SPACE WITH UNDERSCORE FOR COLUMNS
df_country.columns = df_country.columns.str.replace(' ', '_')

#LOWERCASE ALL THE COLUMN NAMES
df_country.columns = [x.lower() for x in df_country.columns]


#REMOVE UNWANTED CHARACTERS FROM ROWS
df_country.country_name = df_country.country_name.apply(lambda x: stripChar(x))

#Capitalize
df_country.country_name=df_country.country_name.apply(lambda x: x.title())

#REMOVE DUPLICATES
df_country = df_country.drop_duplicates(keep='first')

#Not empty
#df_country = df_country[df_country['country_code'].str.contains('[a-zA-Z]$')]
df_country = df_country.dropna()

df_country

#WRITING THE TABLE TO DATABASE
df_country.to_sql(con=dbConnection,name='COUNTRY',if_exists='append',index=False)

display(df_country)
print("Country table added to database successfully")

,country_name,country_code
0,Afghanistan,AF
1,Ã…Land Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS
...,...,...
244,Wallis And Futuna,WF
245,Western Sahara,EH
246,Yemen,YE
247,Zambia,ZM


Country table added to database successfully


In [260]:
#POPULATE COUNTRY AREA 

df_country_area = merge_df[['country_code','world_area_code','area_code_id']]
df_country_area = df_country_area.drop_duplicates(keep='first')
#merge_df_ext = merge_df_ext.rename(columns={'Country Abbrev.':'country_code','World Area Code':'world_area_code'})
#df_country['area_code_id'] = df_country.reset_index()

df_country_area = df_country_area[df_country_area['country_code'].str.contains('[a-zA-Z]$')]

# merge_df_ext.reset_index(inplace=True)
# merge_df_ext.index = merge_df_ext.index + 1
# merge_df_ext['area_code_id'] = merge_df_ext.index
# merge_df_ext = merge_df_ext.drop(['index'],axis=1)
# merge_df_ext

#WRITING THE TABLE TO DATABASE
df = pd.merge(df_country,df_country_area,how="left",on=['country_code'])
df = df.dropna(how='any')
df

df_country_area_final = df[['area_code_id','country_code','world_area_code']]


#WRITING THE TABLE TO DATABASE
df_country_area_final.to_sql(con=dbConnection,name='COUNTRY_AREA',if_exists='append',index=False)

display(df_country_area_final)
print("COUNTRY_AREA table added to database successfully")
len(list(df['area_code_id'].unique()))

,area_code_id,country_code,world_area_code
0,AF701,AF,701
2,AL0,AL,0
3,DZ500,DZ,500
4,AS5,AS,5
6,AO502,AO,502
...,...,...,...
273,VI4,VI,4
274,WF890,WF,890
276,YE694,YE,694
277,ZM597,ZM,597


COUNTRY_AREA table added to database successfully


247

In [261]:
##POPULATE CITY TABLE

df_city = merge_df[['iata_airport','city_name','country_code']]
df_city = df_city.drop_duplicates(keep='first')
df_city=df_city.rename(columns={'iata_airport':'city_code'})
df_city['city_code'] = df_city['city_code'].str.upper()

df_city = df_city.drop_duplicates(keep='first')
df_city

df_city = df_city[df_city['country_code'].str.contains('[a-zA-Z]$')]


#WRITING THE TABLE TO DATABASE

df_country_code = df_country[['country_code']]
df = pd.merge(df_country_code,df_city,how="left",on=['country_code'])
df.drop_duplicates(subset = ['city_code'], keep = 'first', inplace = True) 
df = df.dropna(how='any')

df = df[['city_code','city_name','country_code']]
df_city_final=df
display(df_city_final)


#WRITING THE TABLE TO DATABASE
df_city_final.to_sql(con=dbConnection,name='CITY',if_exists='append',index=False)

print("City table added to database successfully")
len(list(df_city_final['city_code'].unique()))

,city_code,city_name,country_code
0,FAH,Farah,AF
1,MMZ,Maimana,AF
2,KWH,Khwahan,AF
3,SBF,Sardeh Band,AF
4,KBL,Kabul,AF
...,...,...,...
7279,GWE,Gweru,ZW
7280,BUQ,Bulawayo,ZW
7281,CHJ,Chipinge,ZW
7282,KAB,Kariba,ZW


City table added to database successfully


7252

In [262]:
#POPULATE AIRPORT TABLE

df_airport = merge_df[['iata_airport','icao_airport','airport_name','state','latitude','longitude','area_code_id']]
df_airport = df_airport.drop_duplicates(keep='first')
df_airport
df_airport=df_airport.rename(columns={'iata_airport':'city_code'})

df_city_code = df_city_final[['city_code']]
df = pd.merge(df_city_code,df_airport,how="left",on=['city_code'])
df.drop_duplicates(subset = ['city_code'], keep = 'first', inplace = True) 
df = df.dropna(how='any')
df=df.rename(columns={'city_code':'iata_airport','icao_code':'icao_airport'})
df

# df = df[['city_code','city_name','country_code']]
# df_city_final=df
# display(df_city_final)

#WRITING THE TABLE TO DATABASE
df.to_sql(con=dbConnection,name='AIRPORT',if_exists='replace',index=False)

display(df)
print("AIRPORT table added to database successfully")

len(list(df['iata_airport'].unique()))


,iata_airport,icao_airport,airport_name,state,latitude,longitude,area_code_id
0,FAH,OAFR,Farah Airport,Farah,32.367001,62.182999,AF701
1,MMZ,OAMN,Maymana Airport,Faryab,35.930801,64.760902,AF701
2,KWH,OAHN,Khwahan Airport,Badakhshan,37.882999,70.217003,AF701
3,SBF,OADS,Sardeh Band Airport,Ghazni,33.320702,68.636498,AF701
4,KBL,OAKB,Hamid Karzai International Airport,Kabul,34.565899,69.212303,AF701
...,...,...,...,...,...,...,...
7247,GWE,FVTL,Thornhill Air Base,Midlands,-19.436399,29.861900,ZW565
7248,BUQ,FVBU,Joshua Mqabuko Nkomo International Airport,Bulawayo,-20.017401,28.617901,ZW565
7249,CHJ,FVCH,Chipinge Airport,Manicaland,-20.206699,32.628300,ZW565
7250,KAB,FVKB,Kariba Airport,Mashonaland West,-16.519800,28.885000,ZW565


AIRPORT table added to database successfully


7252

In [263]:
#POPULATE AIRLINE TABLE

df_airlines_three = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Data/Airlines/airlines-data-02.csv",header=None, names=['id','name','alias','iata','icao','callsign','country','active'])
df_airlines_three = df_airlines_three.drop([0])
df_airlines_three = df_airlines_three.drop([1])
df_airlines_three = df_airlines_three.drop([2])
df_airlines_three = df_airlines_three.drop([3])

#REMOVING SPACES
df_airlines_three=df_airlines_three.rename(columns=lambda x: x.strip())

#LOWERCASE COLUMN NAMES
df_airlines_three.columns = [x.lower() for x in df_airlines_three.columns]

df_airlines_three=df_airlines_three.drop(['id','alias','callsign'],axis=1)


#RENAME COLUMNS
df_airlines_three.rename(columns={'name':'airline_name','iata':'iata_airline','icao':'icao_airline','country':'country_name'}, inplace=True)

df_airlines_three = df_airlines_three.dropna()

df_airlines_three = df_airlines_three[(df_airlines_three.iata_airline != "\\N")]
df_airlines_three = df_airlines_three[(df_airlines_three.country_name != "\\N")]
df_airlines_three = df_airlines_three[(df_airlines_three.icao_airline != "\\N")]

df_airlines_three = df_airlines_three[df_airlines_three['iata_airline'].str.contains('[a-zA-Z]$')]
df_airlines_three = df_airlines_three[df_airlines_three['icao_airline'].str.contains('[a-zA-Z]$')]

df_airlines_three = df_airlines_three.drop_duplicates(keep='first')
#Removing duplicate iata values
df_airlines_three.drop_duplicates(subset = ['iata_airline'], keep = 'first', inplace = True) 

#df_airlines_three

df_airlines_merge = pd.merge(df_airlines_three, df_country, on='country_name', how='inner') 

In [264]:
#POPULATE COUNTRY AIRLINE TABLE
df_country_airline = df_airlines_merge[['iata_airline','country_code']]
df_country_airline = df_country_airline.drop_duplicates(keep='first')
df_country_airline['country_airline_id'] = df_country_airline['iata_airline']+df_country_airline['country_code']

df_country_airline = df_country_airline[['country_airline_id','iata_airline','country_code']]
df_country_airline

#WRITING THE TABLE TO DATABASE
df_country_airline.to_sql(con=dbConnection,name='AIRLINE_COUNTRY_DETAILS',if_exists='append',index=False)

display(df_country_airline)
print("AIRLINE_COUNTRY_DETAILS table added to database successfully")

,country_airline_id,iata_airline,country_code
0,1TZA,1T,ZA
1,MNZA,MN,ZA
2,IDZA,ID,ZA
3,JEZA,JE,ZA
4,CEZA,CE,ZA
...,...,...,...
667,IYYE,IY,YE
668,FUYE,FU,YE
669,MLMV,ML,MV
670,DNSN,DN,SN


AIRLINE_COUNTRY_DETAILS table added to database successfully


In [265]:
#POPULATE AIRLINE TABLE

df_airline = df_airlines_merge[['iata_airline','icao_airline','airline_name','active']]
df_airline = df_airline.drop_duplicates(keep='first')
# df_airline


#df_airline=df_airline.drop_duplicates(subset = ['iata_airline'], keep = 'first', inplace = True) 
df_airline_final = df_airline


#WRITING THE TABLE TO DATABASE
df_airline_final.to_sql(con=dbConnection,name='AIRLINES',if_exists='append',index=False)

display(df_airline_final)
print("AIRLINES table added to database successfully")
len(list(df_country_airline['country_airline_id'].unique()))

,iata_airline,icao_airline,airline_name,active
0,1T,RNX,1Time Airline,Y
1,MN,CAW,Comair,Y
2,ID,ITK,Interlink Airlines,Y
3,JE,MNO,Mango,Y
4,CE,NTW,Nationwide Airlines,Y
...,...,...,...,...
667,IY,IYE,Yemenia,Y
668,FU,FXX,Felix Airways,Y
669,ML,MAV,Maldivo Airlines,Y
670,DN,SGG,Senegal Airlines,Y


AIRLINES table added to database successfully


672

In [266]:
#AIRLINE REVIEWS

#Reading the Airline CSV files
def readData():
    colnames = ['airline_name','overall_score','review_title','reviewer_country','review_date','aircraft_model','travel_type','seat_type','route','date_flown','seat_comfort_rating','service_rating','food_rating','entertainment_rating','groundservice_rating','wifi_rating','value_rating','recommended']
    Afrance = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/air-france.csv",header = None,names = colnames)
    Ana = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/ana-all-nippon-airways.csv",header = None,names = colnames)
    Emirates = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/emirates.csv",header = None,names = colnames)
    Japan = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/japan-airlines.csv",header = None,names = colnames)
    Korean = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/korean-air.csv",header = None,names = colnames)
    Qantas = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/qantas-airways.csv",header = None,names = colnames)
    Qatar = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/qatar-airways.csv",header = None,names = colnames)
    Singapore = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/singapore-airlines.csv",header = None,names = colnames)
    Swiss = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/swiss-international-air-lines.csv",header = None,names = colnames)
    Turkish = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/turkish-airlines.csv",header = None,names = colnames)
    df = pd.concat([Afrance,Afrance,Ana,Emirates,Japan,Korean,Qantas,Qatar,Singapore,Swiss,Turkish])
    return df

In [267]:
#READ DATA
df_airline_reviews = readData()
df_airline_reviews

#DROP DUPLICATES
df_airline_reviews = df_airline_reviews.drop_duplicates(keep='first')

#DROP AIRLINE_REVIEWS ROW 0
df_airline_reviews = df_airline_reviews.drop([0])


#STRIP CHARACTERS
df_airline_reviews.review_title = df_airline_reviews.review_title.apply(lambda x: stripChar(x))
df_airline_reviews.reviewer_country = df_airline_reviews.reviewer_country.apply(lambda x: stripChar(x))
#display(df_airline_reviews)


#FILL EMPTY CELLS WITH VAL 0
df_airline_reviews.recommended = df_airline_reviews.recommended.fillna(value = 0)
#display(df_airline_reviews)

#FILL 1 TO CELLS WITH VAL YES
df_airline_reviews.recommended = df_airline_reviews.recommended.eq('yes').mul(1)
#display(df_airline_reviews)

#DATETIME CHANGES
df_airline_reviews['review_date'] = pd.to_datetime(df_airline_reviews['review_date'])
# df_airline_reviews['year'] = df_airline_reviews['review_date'].dt.year
# df_airline_reviews['month'] = df_airline_reviews['review_date'].dt.month
# df_airline_reviews.drop(['date_flown'], axis=1)


#display(df_airline_reviews)


In [268]:
df_new = df_airline_reviews[['airline_name']]
df_new['value']=df_airline_reviews.airline_name.str.lower().isin(list(df_airline_final.airline_name.str.lower()))
df_new=df_new.loc[df_new['value'].isin([True])]
df_new=df_new.drop_duplicates(keep='first')
#df_new

df_airline_final['airline_name'] = df_airline_final['airline_name'].str.lower()
df_airline_final

df_airlines_iata = df_airline_final[['airline_name','iata_airline']]
df_airlines_iata

#MERGING 
merge_df = pd.merge(df_airline_reviews, df_airlines_iata, on='airline_name', how='left')
df_review_main = merge_df
#df_review_main

/var/folders/c9/wytgs3g14698n1_9r5l9959c0000gn/T/ipykernel_10746/906472037.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [269]:
##POPULATE AIRLINE_REVIEWS TABLE

df_review_final = df_review_main[['iata_airline','review_title','review_date','seat_comfort_rating','service_rating','food_rating','entertainment_rating','groundservice_rating','wifi_rating','value_rating','overall_score','recommended']]
df_review_final = df_review_final.drop_duplicates(keep='first')
df_review_final


#WRITING THE TABLE TO DATABASE
df_review_final.to_sql(con=dbConnection,name='AIRLINE_REVIEWS',if_exists='replace',index=False)

display(df_country_airline)
print("AIRLINE_REVIEWS table added to database successfully")

,country_airline_id,iata_airline,country_code
0,1TZA,1T,ZA
1,MNZA,MN,ZA
2,IDZA,ID,ZA
3,JEZA,JE,ZA
4,CEZA,CE,ZA
...,...,...,...
667,IYYE,IY,YE
668,FUYE,FU,YE
669,MLMV,ML,MV
670,DNSN,DN,SN


AIRLINE_REVIEWS table added to database successfully
